In [11]:
import cv2
from ultralytics import YOLO
from ultralytics.solutions import object_counter
from queue import Queue
from threading import Thread

def read_frames(cap, queue, frame_skip):
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        if frame_count % frame_skip == 0:
            queue.put(frame)
    queue.put(None)  # Signal end of video

def count_vehicles(video_path, model_path, frame_skip=2, batch_size=4):
    # Initialize YOLO model
    model = YOLO(model_path)

    # Open video capture
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"

    # Initialize object counter with parameters
    counter = object_counter.ObjectCounter(
        region=[(20, 400), (1080, 400)],
        show_in=True,
        show_out=True,
        up_angle=145.0,
        down_angle=90,
        kpts=[6, 8, 10],
        analytics_type='line',
        show=False,  # Set to False to avoid displaying frames
        classes=[0, 2, 3, 5, 7]  # Adjust these class indices based on your model
    )

    # Set up frame reading thread
    frame_queue = Queue(maxsize=30)
    read_thread = Thread(target=read_frames, args=(cap, frame_queue, frame_skip))
    read_thread.start()

    frames = []
    while True:
        frame = frame_queue.get()
        if frame is None:
            break
        frames.append(frame)
        
        if len(frames) == batch_size:
            # Process batch
            results = model.track(frames, persist=True)
            
            for i, result in enumerate(results):
                # Process the results with the counter
                counter.count(frames[i])
            
            frames = []

    # Process any remaining frames
    if frames:
        results = model.track(frames, persist=True)
        for i, result in enumerate(results):
            counter.count(frames[i])

    # Release resources
    cap.release()

    # Print final counts
    print("Final counts:")
    print("In count:", counter.in_count)
    print("Out count:", counter.out_count)

    # Print class-wise counts if available
    if hasattr(counter, 'classwise_counts'):
        print("\nClass-wise counts:")
        for class_name, counts in counter.classwise_counts.items():
            print(f"{class_name}: In = {counts['IN']}, Out = {counts['OUT']}")
    else:
        print("\nClass-wise counts are not available.")

# Run the function
count_vehicles("E:\Company\Projects\\1.mp4", "E:\Company\Projects\yolo11n.pt", frame_skip=2, batch_size=4)

Ultralytics Solutions:  {'region': [(20, 400), (1080, 400)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'show': False, 'model': 'E:\\Company\\Projects\\yolo11n.pt', 'classes': [0, 2, 3, 5, 7]}

0: 384x640 1 person, 12 cars, 1 bus, 3 trucks, 107.6ms
1: 384x640 12 cars, 1 bus, 3 trucks, 107.6ms
2: 384x640 13 cars, 1 bus, 2 trucks, 107.6ms
3: 384x640 13 cars, 1 bus, 2 trucks, 107.6ms
Speed: 7.1ms preprocess, 107.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 12 cars, 1 bus, 3 trucks, 100.9ms
Speed: 2.2ms preprocess, 100.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 bus, 3 trucks, 85.4ms
Speed: 2.6ms preprocess, 85.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 bus, 2 trucks, 69.0ms
Speed: 2.0ms preprocess, 69.0ms inference, 0.0ms postprocess per 